#Setup for the syntax tree

In [42]:
import nltk
from nltk import Nonterminal, nonterminals, Production, CFG

nltk.download('punkt')
nltk.download('words')
nltk.download('treebank')

from typing import NamedTuple
import re

[nltk_data] Downloading package punkt to /home/felix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/felix/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /home/felix/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


# Usage example

At least 13 productions

# Setup for Java

Regex tokenizer

In [43]:
class JavaToken(NamedTuple):
  type: str
  value: str
  line: int
  col: int

def tokenize(code):
  reserved_words = {
      "NUMTYPE" : {"int" : 1, "double" : 2, "double" :3, "short" : 4, "float" :5},
      "TYPES": { "enum" :1, "char" :2 , "String" : 3, "boolean" :4},
      "KEYWORDS": {"public", "static", "void", "main", "abstract", "continue", "for", "new", "switch","assert" , "default", "package", "synchronized", "do", "goto", "private", "this","break" ,  "implements", "throw", "byte", "import", "throws","case" ,  "instanceof", "return", "transient","catch" , "extends",  "try", "final", "interface", "static","class", "finally",  "strictfp", "volatile","const",  "native", "super", "while","_"}
  }

  token_specification = [
        ('NOT',   r'\!'),  
        ('PIPE',   r'\|'),  
        ('AMPERSAND',   r'\&'),  
        ('TRUE',   r'true'),  
        ('FALSE',   r'false'),  
        ('POW',   r'\^'),  
        ('DIV',   r'\/'),  
        ('MULT',   r'\*'),  
        ('MINUS',   r'\-'),  
        ('PLUS',   r'\+'),  
        ('QUOT',   r'\"'),  
        ('DOT',   r'\.'),  # Dot
        ('L_BRKT',   r'\['),  # Left bracket
        ('R_BRKT',   r'\]'),  # Right bracket
        ('L_PAR',   r'\('),  # Left parenthesis
        ('R_PAR',   r'\)'),  # Right parenthesis
        ('L_CUR',   r'\{'),  # Left curly
        ('R_CUR',   r'\}'),  # Right curly
        ('NUMBER',   r'\d+(\.\d*)?'),  # Number
        ('ASSIGN',   r'='),            # Assignment operator
        ('LESS',     r'[\<]'),       # Comparison operators
        ('MORE',     r'[\>]'),       # Comparison operators
        ('END',      r';'),            # Statement terminator
        ('ID',       r'\$*[\$_a-zA-Z]+[\$_a-zA-Z\d]*\$*'),    # Identifiers
        ('SKIP',     r'\s+'),    # Skip over spaces, tabs and nl
        ('MISMATCH', r'.'),            # Any other character
    ]
  tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
  line_num = 1
  line_start = 0
  for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group()
        column = mo.start() - line_start
        if kind == 'NUMBER':
          if '.' in value:
            value = float(value)
          else:
            value = int(value)
        elif value in reserved_words['NUMTYPE']:
          kind = 'NUMTYPE'
        elif value in reserved_words['KEYWORDS']:
            kind = value
        elif value in reserved_words['TYPES']:
            kind = value
        elif kind == 'ID' and value == 'Main':
          kind = 'Main'
        elif kind == 'ID' and value == 'main':
          kind = 'main'
        elif kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
            continue
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')
        yield JavaToken(kind, value, line_num, column)

Java Parser

In [44]:
grammar = CFG.fromstring("""
Main -> 'public' 'static' 'void' 'main' 'L_PAR' 'R_PAR' Main_scope
Main_scope -> 'L_CUR' Scope 'R_CUR'

Scope -> Scope_prime
Scope -> Statements
Scope_prime -> 'L_CUR' Statements 'R_CUR'

Statements -> Statement Statements_prime
Statements_prime -> Statements
Statements_prime -> Scope_prime
Statements_prime ->

Statement -> 'END'
Statement -> Expression 'END'
Statement -> Num_Assignment 'END'
Statement -> String_Assignment 'END'

Expression -> Term Expression_prime

Expression_prime -> Operator Term Expression_prime

Expression_prime -> Comp_Operator Term Expression_prime
Expression_prime -> 

Term -> 'L_PAR' Expression 'R_PAR'
Term -> 'NUMBER'
Term -> 'ID'
Term -> 'TRUE'
Term -> 'FALSE'

Comp_Operator -> 'AMPERSAND' 'AMPERSAND'
Comp_Operator -> 'PIPE' 'PIPE'
Comp_Operator -> 'ASSIGN' 'ASSIGN'
Comp_Operator -> 'NOT' 'ASSIGN'
Comp_Operator -> 'MORE'
Comp_Operator -> 'MORE' 'ASSIGN'
Comp_Operator -> 'LESS'
Comp_Operator -> 'LESS' 'ASSIGN'

Operator -> 'POW'
Operator -> 'PLUS'
Operator -> 'MINUS'
Operator -> 'DIV'
Operator -> 'MULT'


Num_Assignment -> 'NUMTYPE' 'ID' 'ASSIGN' Expression
String_Assignment -> 'String' 'ID' 'ASSIGN' String_like
String_like -> 'QUOT' Valid_Chars 'QUOT'

Valid_Chars -> Char Valid_Chars_prime
Valid_Chars_prime -> Valid_Chars
Valid_Chars_prime -> 

Char -> 'ID'
""")

# Create a parser with the defined grammar
parser = nltk.ChartParser(grammar)
grammar.productions()

[Main -> 'public' 'static' 'void' 'main' 'L_PAR' 'R_PAR' Main_scope,
 Main_scope -> 'L_CUR' Scope 'R_CUR',
 Scope -> Scope_prime,
 Scope -> Statements,
 Scope_prime -> 'L_CUR' Statements 'R_CUR',
 Statements -> Statement Statements_prime,
 Statements_prime -> Statements,
 Statements_prime -> Scope_prime,
 Statements_prime -> ,
 Statement -> 'END',
 Statement -> Expression 'END',
 Statement -> Num_Assignment 'END',
 Statement -> String_Assignment 'END',
 Expression -> Term Expression_prime,
 Expression_prime -> Operator Term Expression_prime,
 Expression_prime -> Comp_Operator Term Expression_prime,
 Expression_prime -> ,
 Term -> 'L_PAR' Expression 'R_PAR',
 Term -> 'NUMBER',
 Term -> 'ID',
 Term -> 'TRUE',
 Term -> 'FALSE',
 Comp_Operator -> 'AMPERSAND' 'AMPERSAND',
 Comp_Operator -> 'PIPE' 'PIPE',
 Comp_Operator -> 'ASSIGN' 'ASSIGN',
 Comp_Operator -> 'NOT' 'ASSIGN',
 Comp_Operator -> 'MORE',
 Comp_Operator -> 'MORE' 'ASSIGN',
 Comp_Operator -> 'LESS',
 Comp_Operator -> 'LESS' 'ASSIG

In [45]:
test_sentences = [
    '''public static void main ( ) { ; { ; { String id = "thing" ; } } } ''',
'''public static void main ( ) { String id = " " ; } ''',
'''public static void main ( ) { int id = number * number ; }''',
'''public static void main ( ) { ; } ''',
'''
public static void main ( ) { float id = number ; { String id = " char " ; int id = number + id ; number != id ; ( id >= id ) ; } }
'''
]
tokens = tokenize(test_sentences[0])
t_list = []
for t in tokens:
    print(t.type, t.value)
    t_list.append(t.type)

public public
static static
void void
main main
L_PAR (
R_PAR )
L_CUR {
END ;
L_CUR {
END ;
L_CUR {
String String
ID id
ASSIGN =
QUOT "
ID thing
QUOT "
END ;
R_CUR }
R_CUR }
R_CUR }


In [46]:
for tree in parser.parse(t_list):
    tree.pretty_print()

                   Main                                                                                                                                                                                                       
   _________________|_______________________________                                                                                                                                                                           
  |      |     |    |     |     |               Main_scope                                                                                                                                                                    
  |      |     |    |     |     |      _____________|______________________                                                                                                                                                    
  |      |     |    |     |     |     |     |                            Scope                            